In [8]:
real_window_size_pos = 3
real_window_size_np_chunk = 1
real_window_size_n_dep = 5
real_window_size_depth = 5
real_window_size_n_siblings = 5 

using_pos = 1
using_chunking = 1
using_parsing = 1
using_dependency = 1
#################################################################################################################
window_size_pos = (real_window_size_pos * 2) + 1
window_size_np_chunk = (real_window_size_np_chunk * 2) + 1
window_size_n_dep = (real_window_size_n_dep * 2) + 1
window_size_depth = (real_window_size_depth * 2) + 1
window_size_n_siblings = (real_window_size_n_siblings * 2) + 1


######################################################################################################################
import numpy as np
import nltk
from nltk import pos_tag
from nltk.tokenize.moses import MosesDetokenizer
from data_handler import *
from feature_design_for_spaCy import *
import spacy
nlp = spacy.load('en')
#from spacy.en import English
#parser = English()
detokenizer = MosesDetokenizer()

pre_X_train = load('X_train') 
pre_X_test = load('X_test') 
Y_train = load('Y_train')


### FE2_X_train와 FE2_X_test 초기화
FE4_X_train = []
for sentence in (pre_X_train):
    temp = []
    for token in (sentence):
        temp.append([])
    FE4_X_train.append(temp)

FE4_X_test = []
for sentence in (pre_X_test):
    temp = []
    for token in (sentence):
        temp.append([])
    FE4_X_test.append(temp)
    
### For POS look-up table    
list_spacy_pos = ['DET', 'ADJ', 'NOUN', 'VERB', 'PUNCT', 'CONJ', 'ADP', 'PART', 'ADV', 'PRON', 'NUM', 'PROPN', 'INTJ', 'X', 'SYM']
lookup_table_spacy_pos = { }
for i, token in enumerate(list_spacy_pos):
    lookup_table_spacy_pos[token] = i # POS에서는 1부터(i+1) 시작안해도 된다. POS에는 unknown이 없기 때문이다.
# Add unknown token
print('len(lookup_table_POS) = ', len(lookup_table_spacy_pos))

### For dependent look-up table
list_dependent = [
    'det','amod','compound','nsubj','aux','ROOT','dobj','punct','appos','conj','cc','prep','pobj','pcomp','nsubjpass',\
    'auxpass','xcomp','advmod','advcl','acl','neg','relcl','poss','mark','ccomp','nummod','agent','acomp','expl','attr',\
    'intj','oprd','preconj','nmod','npadvmod','prt','csubj','quantmod','case','predet','dep','dative','parataxis']
lookup_table_dependent = { }
for i, token in enumerate(list_dependent):
    lookup_table_dependent[token] = i
print('len(lookup_table_dependent) = ', len(lookup_table_dependent))

len(lookup_table_POS) =  15
len(lookup_table_dependent) =  43


## POS

In [9]:
print('Before POS:', len(FE4_X_train[0][0]))
#####################
""" Training Data """
#####################
for i, sentence in enumerate(pre_X_train):
    
    detokenized_sent = detokenizer.detokenize(sentence, return_str=True)
    ### for exception [START]
    list_detokenized_sent = list(detokenized_sent)
    if list_detokenized_sent[-1] == '.' and list_detokenized_sent[-2].isupper() == True:
        list_detokenized_sent[-1] = ' .'
    new_detokenized_sent = "".join(list_detokenized_sent)
    ### for exception [END]
    spacy_sent = nlp(new_detokenized_sent)
    

    #print(len(sentence), len(spacy_sent))
    #print(sentence)
    #print(spacy_sent)
    assert(len(sentence)==len(spacy_sent))
    
    
    ##################################################################################################
    for j, token in enumerate(spacy_sent):
       
        FE4_X_train[i][j] += FE_spaCy_POS(j, spacy_sent, lookup_table_spacy_pos, window_size_pos)
        


####################
""" Testing Data """
####################      
for i, sentence in enumerate(pre_X_test):
    
    detokenized_sent = detokenizer.detokenize(sentence, return_str=True)
    ### for exception [START]
    list_detokenized_sent = list(detokenized_sent)
    if list_detokenized_sent[-1] == '.' and list_detokenized_sent[-2].isupper() == True:
        list_detokenized_sent[-1] = ' .'
    new_detokenized_sent = "".join(list_detokenized_sent)
    ### for exception [END]
    spacy_sent = nlp(new_detokenized_sent)
    
    
    assert(len(sentence)==len(spacy_sent))
    
      
    ##################################################################################################
    for j, token in enumerate(spacy_sent):
       
        FE4_X_test[i][j] += FE_spaCy_POS(j, spacy_sent, lookup_table_spacy_pos, window_size_pos) 
        
print('After POS:', len(FE4_X_train[0][0]))

Before POS: 0
After POS: 105


## NP Chuck Position
* == 한 문장에서 나온 NP Chunk List에서 첫 번재 NP Chunk인지 아닌지
* == 한 문장에서 나온 NP Chunk List에서 마지막 NP Chunk인지 아닌지
* ==== NP인지 아닌지
* ==== ROOT에서 오른쪽에 있는 NP인지
* ==== ROOT에서 왼쪽에 있는 NP인지
* ==== ROOT에서 첫 번째 오른쪽에 있는 NP인지
* ====== PP인지 아닌지 (PP 정의: NP앞에 전치사가 있다면 PP이다.)
* ====== PP라면 어떤 종류의 PP인지??? (e.g. for, to, with, ...)
* ====== ROOT에서 오른쪽에 있는 PP인지
* ====== ROOT에서 왼쪽에 있는 PP인지
* ====== ROOT에서 첫 번째 오른쪽에 있는 PP인지
* ======== PP wich right NP 인지 아닌지 (PP 정의: NP앞에 전치사가 있다면 PP이다.)
* ======== ROOT에서 오른쪽에 있는 PP Chunk wich right NP인지
* ======== ROOT에서 왼쪽에 있는 PP Chunk wich right NP인지
* ======== ROOT에서 첫 번째 오른쪽에 있는 PP wich right NP인지
<br>

* =======================================================================
< 더 구현해야될 부분들... >
* NP Chunk가 ROOT 바로 왼쪽에 있는지 (즉, ROOT 사이 또 다른 NP가 없는지)
* NPPP 쌍을 PP기준으로만 구현했는데, NP기준으로도 해야되나...?

* PP가 동사바로 오른쪽 옆에 있는지 아닌지
* PP가 동사바로 오른쪽 옆, 옆에 있는지 아닌지
* PP가 ROOT 바로 옆 (오른쪽/왼쪽) 에 있는지
* PP가 또 다른 PP 오른쪽 옆에 있는지 아닌지 (또는 왼쪽 옆에 있는지 아닌지)


<br>
* second root를 정의해볼까??? (일단은 보류)

In [10]:
# for i, sentence in enumerate(pre_X_train):
    
#     detokenized_sent = detokenizer.detokenize(sentence, return_str=True)
#     ### for exception [START]
#     list_detokenized_sent = list(detokenized_sent)
#     if list_detokenized_sent[-1] == '.' and list_detokenized_sent[-2].isupper() == True:
#         list_detokenized_sent[-1] = ' .'
#     new_detokenized_sent = "".join(list_detokenized_sent)
#     ### for exception [END]
#     spacy_sent = nlp(new_detokenized_sent)
#     assert(len(sentence)==len(spacy_sent))
#     npchunk_numbered_sent = numbering_npchunk(spacy_sent)
#     ppchunk_numbered_sent = numbering_ppchunk(npchunk_numbered_sent, spacy_sent)
    
#     whichprep_sent = numbering_which_ppchunk(ppchunk_numbered_sent, spacy_sent)
    
    
#     for j, token in enumerate(spacy_sent):
#         print(whichprep_sent[j])
        
#     print('==================================================================')
    
# #     print(spacy_sent)
# #     print(npchunk_numbered_sent)
# #     print(ppchunk_numbered_sent)
# #     print(whichprep_sent)
# #     print('\n')
    

    
    
    
    
# #     pp_isjustright_fromnp_numbered_sent = numbering_pp_isJustRight_fromNP_sent(npchunk_numbered_sent, ppchunk_numbered_sent)
# #     ### For just right from Root 
# #     firstNP_fromroot_sent = Xchunk_isFirstRight_fromRoot(npchunk_numbered_sent, spacy_sent)
# #     firstPP_fromroot_sent = Xchunk_isFirstRight_fromRoot(ppchunk_numbered_sent, spacy_sent)
# #     firstPPjustrightNP_fromroot_sent = Xchunk_isFirstRight_fromRoot(pp_isjustright_fromnp_numbered_sent, spacy_sent)
    
# #     print(sentence)
# #     print(npchunk_numbered_sent)
# #     print(ppchunk_numbered_sent)
# #     print(pp_isjustright_fromnp_numbered_sent)
# #     print(firstNP_fromroot_sent)
# #     print(firstPP_fromroot_sent)
# #     print(firstPPjustrightNP_fromroot_sent)
#     ####################
#     #for j, token in enumerate(spacy_sent):
        

In [11]:
print('Before POS:', len(FE4_X_train[0][0]))

#####################
""" Training Data """
##################### 
for i, sentence in enumerate(pre_X_train):
    
    detokenized_sent = detokenizer.detokenize(sentence, return_str=True)
    ### for exception [START]
    list_detokenized_sent = list(detokenized_sent)
    if list_detokenized_sent[-1] == '.' and list_detokenized_sent[-2].isupper() == True:
        list_detokenized_sent[-1] = ' .'
    new_detokenized_sent = "".join(list_detokenized_sent)
    ### for exception [END]
    spacy_sent = nlp(new_detokenized_sent)
    assert(len(sentence)==len(spacy_sent))
    npchunk_numbered_sent = numbering_npchunk(spacy_sent)
    ppchunk_numbered_sent = numbering_ppchunk(npchunk_numbered_sent, spacy_sent)
    pp_isjustright_fromnp_numbered_sent = numbering_pp_isJustRight_fromNP_sent(npchunk_numbered_sent, ppchunk_numbered_sent)
    ### For just right from Root (밑에 3개는 0또는1밖에없다.)
    firstNP_fromroot_sent = Xchunk_isFirstRight_fromRoot(npchunk_numbered_sent, spacy_sent)
    firstPP_fromroot_sent = Xchunk_isFirstRight_fromRoot(ppchunk_numbered_sent, spacy_sent)
    firstPPjustrightNP_fromroot_sent = Xchunk_isFirstRight_fromRoot(pp_isjustright_fromnp_numbered_sent, spacy_sent)
    ## for what is the prep
    whichprep_sent = numbering_which_ppchunk(ppchunk_numbered_sent, spacy_sent)

    ####################
    for j, token in enumerate(spacy_sent):
        
        FE4_X_train[i][j] += is_first_np_chunk(j, npchunk_numbered_sent)
        FE4_X_train[i][j] += is_last_np_chunk(j, npchunk_numbered_sent, spacy_sent)
        FE4_X_train[i][j] += is_np_chuck(j, spacy_sent)
        FE4_X_train[i][j] += np_isLeft_fromRoot(j, npchunk_numbered_sent, spacy_sent)
        FE4_X_train[i][j] += np_isRight_fromRoot(j, npchunk_numbered_sent, spacy_sent)
        FE4_X_train[i][j] += Xchunk_isLeft_fromSubRoot(j, npchunk_numbered_sent, spacy_sent)
        FE4_X_train[i][j] += Xchunk_isRight_fromSubRoot(j, npchunk_numbered_sent, spacy_sent)
        
        # ppchunk_numbered_sent
        FE4_X_train[i][j] += is_Xchunk(j, ppchunk_numbered_sent)
        FE4_X_train[i][j] += Xchunk_isLeft_fromRoot(j, ppchunk_numbered_sent, spacy_sent)
        FE4_X_train[i][j] += Xchunk_isRight_fromRoot(j, ppchunk_numbered_sent, spacy_sent)
        FE4_X_train[i][j] += Xchunk_isLeft_fromSubRoot(j, ppchunk_numbered_sent, spacy_sent)
        FE4_X_train[i][j] += Xchunk_isRight_fromSubRoot(j, ppchunk_numbered_sent, spacy_sent)        
        
        # pp_isjustright_fromnp_numbered_sent
        FE4_X_train[i][j] += is_Xchunk(j, pp_isjustright_fromnp_numbered_sent)
        FE4_X_train[i][j] += Xchunk_isLeft_fromRoot(j, pp_isjustright_fromnp_numbered_sent, spacy_sent)
        FE4_X_train[i][j] += Xchunk_isRight_fromRoot(j, pp_isjustright_fromnp_numbered_sent, spacy_sent)
        
        # For just right from Root
        FE4_X_train[i][j] += is_Xchunk(j, firstNP_fromroot_sent)
        FE4_X_train[i][j] += is_Xchunk(j, firstPP_fromroot_sent)
        FE4_X_train[i][j] += is_Xchunk(j, firstPPjustrightNP_fromroot_sent)
        
        # for what is the prep
        FE4_X_train[i][j] += whichprep_sent[j]
        
####################
""" Testing Data """
#################### 
for i, sentence in enumerate(pre_X_test):
    
    detokenized_sent = detokenizer.detokenize(sentence, return_str=True)
    ### for exception [START]
    list_detokenized_sent = list(detokenized_sent)
    if list_detokenized_sent[-1] == '.' and list_detokenized_sent[-2].isupper() == True:
        list_detokenized_sent[-1] = ' .'
    new_detokenized_sent = "".join(list_detokenized_sent)
    ### for exception [END]
    spacy_sent = nlp(new_detokenized_sent)
    assert(len(sentence)==len(spacy_sent))
    npchunk_numbered_sent = numbering_npchunk(spacy_sent)
    ppchunk_numbered_sent = numbering_ppchunk(npchunk_numbered_sent, spacy_sent)
    pp_isjustright_fromnp_numbered_sent = numbering_pp_isJustRight_fromNP_sent(npchunk_numbered_sent, ppchunk_numbered_sent)    
    assert(len(npchunk_numbered_sent)==len(ppchunk_numbered_sent)==len(pp_isjustright_fromnp_numbered_sent))
    ### For just right from Root
    firstNP_fromroot_sent = Xchunk_isFirstRight_fromRoot(npchunk_numbered_sent, spacy_sent)
    firstPP_fromroot_sent = Xchunk_isFirstRight_fromRoot(ppchunk_numbered_sent, spacy_sent)
    firstPPjustrightNP_fromroot_sent = Xchunk_isFirstRight_fromRoot(pp_isjustright_fromnp_numbered_sent, spacy_sent)
    ## for what is the prep
    whichprep_sent = numbering_which_ppchunk(ppchunk_numbered_sent, spacy_sent)  
    
    
    
    ####################
    for j, token in enumerate(spacy_sent):
        
        FE4_X_test[i][j] += is_first_np_chunk(j, npchunk_numbered_sent)
        FE4_X_test[i][j] += is_last_np_chunk(j, npchunk_numbered_sent, spacy_sent)
        FE4_X_test[i][j] += is_np_chuck(j, spacy_sent)
        FE4_X_test[i][j] += np_isLeft_fromRoot(j, npchunk_numbered_sent, spacy_sent)
        FE4_X_test[i][j] += np_isRight_fromRoot(j, npchunk_numbered_sent, spacy_sent)
        FE4_X_test[i][j] += Xchunk_isLeft_fromSubRoot(j, npchunk_numbered_sent, spacy_sent)
        FE4_X_test[i][j] += Xchunk_isRight_fromSubRoot(j, npchunk_numbered_sent, spacy_sent)
        
        # ppchunk_numbered_sent
        FE4_X_test[i][j] += is_Xchunk(j, ppchunk_numbered_sent)
        FE4_X_test[i][j] += Xchunk_isLeft_fromRoot(j, ppchunk_numbered_sent, spacy_sent)
        FE4_X_test[i][j] += Xchunk_isRight_fromRoot(j, ppchunk_numbered_sent, spacy_sent)
        FE4_X_test[i][j] += Xchunk_isLeft_fromSubRoot(j, ppchunk_numbered_sent, spacy_sent)
        FE4_X_test[i][j] += Xchunk_isRight_fromSubRoot(j, ppchunk_numbered_sent, spacy_sent) 
        
        # pp_isjustright_fromnp_numbered_sent
        FE4_X_test[i][j] += is_Xchunk(j, pp_isjustright_fromnp_numbered_sent)
        FE4_X_test[i][j] += Xchunk_isLeft_fromRoot(j, pp_isjustright_fromnp_numbered_sent, spacy_sent)
        FE4_X_test[i][j] += Xchunk_isRight_fromRoot(j, pp_isjustright_fromnp_numbered_sent, spacy_sent)

        # For just right from Root
        FE4_X_test[i][j] += is_Xchunk(j, firstNP_fromroot_sent)
        FE4_X_test[i][j] += is_Xchunk(j, firstPP_fromroot_sent)
        FE4_X_test[i][j] += is_Xchunk(j, firstPPjustrightNP_fromroot_sent)        
        
        # for what is the prep
        FE4_X_test[i][j] += whichprep_sent[j]        
        

print('After POS:', len(FE4_X_train[0][0]))

# [A clinical lab section, the clinical site name, day, time, the lab]
# A clinical lab section shall include the clinical site name, the class, instructor, day and time of the lab.
# [1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 3, 0, 4, 0, 5, 5, 0]

Before POS: 105
After POS: 134


## Dependency Parsing (Dependency based Parsing Tree)
* aux -> ROOT dependency인지 아닌지...(for sub priority)

<br>
* = ROOT까지 (token단위) 거리(distance) (normalization은 한 문장에서 총 token 개수)
* = ROOT까지 (head단위) 거리(distance): 거리단위는 단어가 아니고, anscestor 개수, (normalization은 한 문장에서 
    * 부사절에 속하는 단어들 또는 수식어구들이 root까지의 거리가 멀다.
* === 자신의 dependency 크기
* === 자신의 dependent 종류
* === token이 ROOT 왼쪽에 있는지 오른쪽에 있는지
* =========== token이 SUB ROOT (VERB) 왼쪽에 있는지 오른쪽에 있는지
* =========== token이 SUB NOUN (NOUN) 왼쪽에 있는지 오른쪽에 있는지
* ===== Head단어의 dependency 크기
* ===== Head단어의 dependent 종류
* ===== Head단어의 pos가 무엇인지
* ======== ROOT까지가는 모든 dependent가 있는 path를 통해서 implictly하게 grouping할 수 있다.
   * high-level의 ancestor의 dependent기준으로 그룹핑을 할 수 있다. (즉, 하위에 있는 dependent이거나 덜 중요한 dependent (compound, amod, appos, ..)들은 고려하지 않는다.
   * 그룹핑 크기가 큰 순서부터 차례대로 그룹핑해준다. (advcl -> acl -> prep -> ...
   * root까지가는 head list 중에서 가장 첫 번째의 verb head가 일치하는 단어들끼리의 묶음 -> implictly하게 sentence segmentation을 할 수 있다.
   * 그 묶음의 동사가 ROOT동사인지 (즉, 주절인지)
   * ROOT로가는 dependent path 길목 중에서 중요한 dependent가 있는지 없는지...
   * voice구별안해도, dependent로 구별할 수 있다. nsubjpass와 nsubj로...





<br>
* =========================== 고려해볼 점들...
* head단어의 단어가 무엇인지 (1-gram lookup table과 연결)
* sentence segmentation (clause단위로 나누기)
* 여러개의 clause가 있다하면, 현재 token이 주절에 속하는지 안속하는지 (가장 높은 dependency subtree개수로 주절로 인식하자.) 그리고 부사절은 앞에서부터 numbering한다.
* 동사의 voice?


In [12]:
for i, sentence in enumerate(pre_X_train):
    detokenized_sent = detokenizer.detokenize(sentence, return_str=True)
    ### for exception [START]
    list_detokenized_sent = list(detokenized_sent)
    if list_detokenized_sent[-1] == '.' and list_detokenized_sent[-2].isupper() == True:
        list_detokenized_sent[-1] = ' .'
    new_detokenized_sent = "".join(list_detokenized_sent)
    ### for exception [END]
    spacy_sent = nlp(new_detokenized_sent)
    assert(len(sentence)==len(spacy_sent))
    # For Normalization
    len_sentence = len(sentence)
    max_len_heads = lenMax_amongHeadList_toRoot(spacy_sent)
    max_len_subtrees = lenMax_amongSubtreeList_toRoot(spacy_sent)

    ####################
    for j, token in enumerate(spacy_sent):
        
        # Distance
        FE4_X_train[i][j] += normalization(len_sentence, 1, j+1)
        FE4_X_train[i][j] += normalization(max_len_heads, 0, len(headList_to_root(token)))
        
        # Current token
        FE4_X_train[i][j] += normalization(max_len_subtrees, 1, len(list(token.subtree)))
        FE4_X_train[i][j] += what_is_myDependent(token, lookup_table_dependent)
        FE4_X_train[i][j] += is_LeftRight_fromRoot(j, spacy_sent)
        
        # Head token
        FE4_X_train[i][j] += normalization(max_len_subtrees, 1, len(list(token.head.subtree)))
        FE4_X_train[i][j] += what_is_headDependent(token, lookup_table_dependent)
        FE4_X_train[i][j] += what_is_headPOS(token, lookup_table_spacy_pos)
        
        # Implicitly Grouping
        headlist_to_root = [(dependent_token.dep_) for dependent_token in headList_to_root(token)]
        headpath_to_root = remove_duplicate(headlist_to_root)        
        FE4_X_train[i][j] += high_level_implicit_grouping(headpath_to_root)
        
        # for sub prioirty
        FE4_X_train[i][j] += is_subpriority(j, spacy_sent)
        
        # SUB ROOT & SUB NOUN
        FE4_X_train[i][j] += is_middle_between_RtAndSubRt(j, spacy_sent)
        FE4_X_train[i][j] += is_Right_fromSubRoot(j, spacy_sent)
        
        ### Ancestors와 children은 패턴이 안보여서 일단 pass
        #print(token, '---' , headList_to_root(token))
        #print(token, len(list(token.subtree)))
        #print(token, '\t', len(list(token.subtree)),len(list(token.ancestors)),len(list(token.children)), '\t\t', list(token.ancestors), '\t\t', list(token.children)  )
        
#     for j, token in enumerate(spacy_sent):    
#         print(token.orth_,'\t', token.dep_,'\t', token.head.orth_, [t.orth_ for t in token.lefts], [t.orth_ for t in token.rights])
   
#     print('-----------------------------------------------------------------------------------------------------')

In [13]:
for i, sentence in enumerate(pre_X_test):
    detokenized_sent = detokenizer.detokenize(sentence, return_str=True)
    ### for exception [START]
    list_detokenized_sent = list(detokenized_sent)
    if list_detokenized_sent[-1] == '.' and list_detokenized_sent[-2].isupper() == True:
        list_detokenized_sent[-1] = ' .'
    new_detokenized_sent = "".join(list_detokenized_sent)
    ### for exception [END]
    spacy_sent = nlp(new_detokenized_sent)
    assert(len(sentence)==len(spacy_sent))
    # For Normalization
    len_sentence = len(sentence)
    max_len_heads = lenMax_amongHeadList_toRoot(spacy_sent)
    max_len_subtrees = lenMax_amongSubtreeList_toRoot(spacy_sent)

    ####################
    for j, token in enumerate(spacy_sent):
        
        # Distance
        FE4_X_test[i][j] += normalization(len_sentence, 1, j+1)
        FE4_X_test[i][j] += normalization(max_len_heads, 0, len(headList_to_root(token)))
        
        # Current token
        FE4_X_test[i][j] += normalization(max_len_subtrees, 1, len(list(token.subtree)))
        FE4_X_test[i][j] += what_is_myDependent(token, lookup_table_dependent)
        FE4_X_test[i][j] += is_LeftRight_fromRoot(j, spacy_sent)
        
        # Head token
        FE4_X_test[i][j] += normalization(max_len_subtrees, 1, len(list(token.head.subtree)))
        FE4_X_test[i][j] += what_is_headDependent(token, lookup_table_dependent)
        FE4_X_test[i][j] += what_is_headPOS(token, lookup_table_spacy_pos)
        
        # Implicitly Grouping
        headlist_to_root = [(dependent_token.dep_) for dependent_token in headList_to_root(token)]
        headpath_to_root = remove_duplicate(headlist_to_root)        
        FE4_X_test[i][j] += high_level_implicit_grouping(headpath_to_root)

        # for sub prioirty
        FE4_X_test[i][j] += is_subpriority(j, spacy_sent)
        
        # SUB ROOT & SUB NOUN
        FE4_X_test[i][j] += is_middle_between_RtAndSubRt(j, spacy_sent)
        FE4_X_test[i][j] += is_Right_fromSubRoot(j, spacy_sent)
        
        ### Ancestors와 children은 패턴이 안보여서 일단 pass
        #print(token, '---' , headList_to_root(token))
        #print(token, len(list(token.subtree)))
        #print(token, '\t', len(list(token.subtree)),len(list(token.ancestors)),len(list(token.children)), '\t\t', list(token.ancestors), '\t\t', list(token.children)  )
        
#     for j, token in enumerate(spacy_sent):    
#         print(token.orth_,'\t', token.dep_,'\t', token.head.orth_, [t.orth_ for t in token.lefts], [t.orth_ for t in token.rights])
   
#     print('-----------------------------------------------------------------------------------------------------')


## Write out files

In [14]:
dump(FE4_X_train, 'FE4_X_train')
dump(FE4_X_test, 'FE4_X_test')
print('===> Total feature length =', len(FE4_X_train[0][0]))

===> Total feature length = 295


In [15]:
# # all you have to do to parse text is this:
# #note: the first time you run spaCy in a file it takes a little while to load up its modules
# doc = nlp('If you have a product like message, as a user I want to be able to give it to him by using them, so that I can remove expired and erroneous of ads.')

# # Let's look at the tokens
# # All you have to do is iterate through the parsedData
# # Each token is an object with lots of different properties
# # A property with an underscore at the end returns the string representation
# # while a property without the underscore returns an index (int) into spaCy's vocabulary
# # The probability estimate is based on counts from a 3 billion word
# # corpus, smoothed using the Simple Good-Turing method.
# for i, token in enumerate(doc):
#     print("original:", token.orth, token.orth_)
#     print("lowercased:", token.lower, token.lower_)
#     print("lemma:", token.lemma, token.lemma_)
#     print("shape:", token.shape, token.shape_)
#     print("prefix:", token.prefix, token.prefix_)
#     print("suffix:", token.suffix, token.suffix_)
#     print("log probability:", token.prob)
#     print("Brown cluster id:", token.cluster)
#     print("TOKEN POS:", token.pos_)
#     print("Word Vector:", token.vector.shape)
#     print("----------------------------------------")


In [16]:
# dependency parsing이 제대로 동작하는 않는 경우도 많다.
# 특히 동사에 따라 dependent가 바뀐다. 구조는 똑같지만,..... 
# if 를 that으로 바꾸면, 어느정도 advcl을 면할 수는 있지만, 특정 동사들에 따라서 advcl를 가지게 되는 경우가 종종 있다. 이경우는 감수해야 될 듯하다.
# Process sentences 'Hello, world. Natural Language Processing in 10 lines of code.' using spaCy
text1 = 'It would be nice to have a feature whereby you can reset the statistics,  but still retain the existing statistics.'
text2 = 'It would be great that there was the ability to configure a Today button to be displayed just under the calendar.'
text3 = 'It would be nice in the future that you could save two or three user generated filename patterns.'
text4 = 'It would be very nice that I could control the TV volume directly in the program.'
text5 = 'It would be great that there was the ability to configure a Today button to be displayed just under the calendar.'
text6 = 'It would be a very nice that I could control the TV volume directly in the program.'
text7 = 'it Would be nice that you could have the width of the indent and use tabs or spaces.'
text8 = 'it Would be nice that you could configure the width of the indent and use tabs or spaces.'

text9 = 'It would be nice to configure a ticker bar at the bottom of the UI window that scrolls clickable track names chosen by the JukeDaddy,  configured in the same way as AutoPlay.'
text10 = 'It would be nice that there were some way to Synchronise/subscribe to my published phpiCalendar with SyncMLcompliant mobile devices.'
text11 = 'It would be nice that you could be an hide option for tabbed document inside a docking container similar to visual studio'
text12 = 'Some sort of mini checkmark would be nice, so that users can chose if they want to certain lines to be excluded/included from a diff.'
text13 = 'It would be nice if the configuration file would allow the admin to give text names to the drop down list.'
text14 = "It would be nice that the focus was automatically set to the text fields in PS and Tex (so that we don't have to click on them to start typing)"
text15 = "It would really be nice that the class XYSeriesCollection would provide a method public Vector getSeries that would return a Vector with XYSeries objects."
text16 = "As an alternative it could be nice that the application could be launched from the terminal with the .csv of .xml file path as a parameter."
text17 = 'As a salesperson, I would like for the UI of the application to remain responsive while the report is loading, so that I can cancel the report if desired.'

t1 = 'The report of needed classes shall include (but not be limited to) classes to be offered, number of sections needed, number of labs needed, and room types needed'
t1 = ''# dependency parsing이 제대로 동작하는 않는 경우도 많다.
# 특히 동사에 따라 dependent가 바뀐다. 구조는 똑같지만,..... 
# if 를 that으로 바꾸면, 어느정도 advcl을 면할 수는 있지만, 특정 동사들에 따라서 advcl를 가지게 되는 경우가 종종 있다. 이경우는 감수해야 될 듯하다.
# Process sentences 'Hello, world. Natural Language Processing in 10 lines of code.' using spaCy
text1 = 'It would be nice to have a feature whereby you can reset the statistics,  but still retain the existing statistics.'
text20 = 'there was the ability to configure a Today button to be displayed just under the calendar.'


text3 = 'It would be nice in the future that you could save two or three user generated filename patterns.'
text4 = 'It would be very nice that I could control the TV volume directly in the program.'
text5 = 'It would be great that there was the ability to configure a Today button to be displayed just under the calendar.'
text6 = 'It would be a very nice that I could control the TV volume directly in the program.'
text7 = 'it Would be nice that you could have the width of the indent and use tabs or spaces.'
text8 = 'it Would be nice that you could configure the width of the indent and use tabs or spaces.'

text9 = 'It would configure a ticker bar at the bottom of the UI window that scrolls clickable track names chosen by the JukeDaddy,  configured in the same way as AutoPlay.'
text10 = 'It would be nice that there were some way to Synchronise/subscribe to my published phpiCalendar with SyncMLcompliant mobile devices.'
text11 = 'It would be nice that you could be an hide option for tabbed document inside a docking container similar to visual studio'
text12 = 'Some sort of mini checkmark would be nice, so that users can chose if they want to certain lines to be excluded/included from a diff.'
text13 = 'It would be nice if the configuration file would allow the admin to give text names to the drop down list.'
text14 = "It would be nice that the focus was automatically set to the text fields in PS and Tex (so that we don't have to click on them to start typing)"
text15 = "It would really be nice that the class XYSeriesCollection would provide a method public Vector getSeries that would return a Vector with XYSeries objects."
text16 = "As an alternative it could be nice that the application could be launched from the terminal with the .csv of .xml file path as a parameter."
text17 = 'As a salesperson, I would like for the UI of the application to remain responsive while the report is loading, so that I can cancel the report if desired.'

t1 = 'The report of needed classes shall include (but not be limited to) classes to be offered, number of sections needed, number of labs needed, and room types needed'
t1 = 'Classes for a given cohort shall not conflict with regards to the time and day that they are offered.'
t1 = "The Disputes System must be beutiful by both internal and external users."

t1 = "The IQA subsystem shall take into account current inventory status and adjust inventory of substitutionary ingredients accordingly."
t1 = 'Product shall allow entering, storing and modifying product menus used by POS terminals.'
t1 = 'PME Subsystem shall allow keeping submenus within menus and products within submenus in a managed and persisted order."'
t1 = "The RMS System shall have Windows XP looking and feeling and use GUI futures making its use intuitive"
h1 = 'The system shall have a MDI form that allows for the viewing of the graph and the data table.'
t1 = 'The system shall offer the ability to pause and resume the refresh of data.'
t1 = 'The Disputes System must prevent users from accessing any dispute cases that do not belong to their cardholder base.'
#t1='The Disputes System must allow the user to create three unique types of ticket retrieval requests.'
#t1="the search functionality must further allow the user to limit the results of the search by a date range, the type of dispute (ticket retrieval request or chargeback notification), the case status (open, closed or all) and the dispute reason code."
#t1="The Disputes System shall provide the users with the ability to view the history of all activities that have occurred on the dispute case."
t1='Website shall allow customers to view reviews of selected movies by other customers'
t1='It would get also Information about user specified packages installed on that system like glibc or apache or mysql etc.'
t1="It would be great to have columns in the Parent and Student interfaces where users could see calendar and web page links added by faculty."
t1="A team member can view the current burndown chart on the status page, and can click it for a larger view."
t1='It would be nice that you could save two or three user generated by filename patterns so that I can easily switch between them'
t1 = 'It would be nice to add a column to mailbox table so that this would be used to store a path to user sieve script'
#t1='As a user I want to be able to manage ads, so that I can remove expired and erroneous ads.'
t1='It would be great that you could add the aspect ratio of the possible resolutions one can choose from.'
t2='As I get cramps in my right hand from holding the mouse, I would like to have some hotkeys so that I can have cramps in my left hand'
t1='is it possible to add a Description column in the Attributes & Operations html table that I can put some comment in my Delphi surce code and Model just pull these comment into the html document?'
t1='It would be nice to have a feature whereby you can reset the statistics, but still retain the existing statistics.'
t1='Combined with tcp/ip stack it could be handy to set a treshold in which someone can specify how many bad clusters is acceptable.'
t1='A Program of Study shall consist of a program name and listing of required classes (both clinical and nonclinical) that must be completed.'
h1='The system shall allow a Program Administrator or Nursing staff member to remove a student from a clinical lab section.'
tt1='allow option to hide tray icon.'
doc = nlp(t2)


# # For every token in document, print it's tokens to the root
# for token in doc:
#     print('{} --> {}'.format(token, headList_to_root(token)))

# Print dependency labels of the tokens
# for token in doc:
#     print(token)
#     print(token.dep_, token.head.dep_, token.head.head.dep_)
# import operator
# verb_withNsubtree = {}


# for i, token in enumerate(doc):

#     if token.pos_ == 'VERB':
#         verb_withNsubtree[i] = len(list(token.subtree))
        
#     #print(token, '---' , headList_to_root(token))
# #     print(token, token.pos_ ,'\t', len(list(token.subtree)))
#     #print(token, '\t', len(list(token.subtree)),len(list(token.ancestors)),len(list(token.children)), '\t\t', list(token.ancestors), '\t\t', list(token.children)  )

# print(verb_withNsubtree)

# verb_withNsubtree = sorted(verb_withNsubtree.items(), key=operator.itemgetter(1), reverse=True) 
# print(verb_withNsubtree)

for token in doc:
    print(token)
    print('-> '.join(['{}-{}'.format(dependent_token, dependent_token.dep_) for dependent_token in headList_to_root(token)]))
    print('\n')
doc = nlp(t1)


# # For every token in document, print it's tokens to the root
# for token in doc:
#     print('{} --> {}'.format(token, headList_to_root(token)))

# Print dependency labels of the tokens
# for token in doc:
#     print(token)
#     print('-> '.join(['{}-{}'.format(dependent_token, dependent_token.dep_) for dependent_token in headList_to_root(token)]))
#     print('\n')

As
As-mark-> get-advcl-> get-advcl-> like-ROOT


I
I-nsubj-> get-advcl-> get-advcl-> like-ROOT


get
get-advcl-> like-ROOT


cramps
cramps-dobj-> get-advcl-> get-advcl-> like-ROOT


in
in-prep-> get-advcl-> get-advcl-> like-ROOT


my
my-poss-> hand-pobj-> hand-pobj-> in-prep-> in-prep-> get-advcl-> get-advcl-> like-ROOT


right
right-amod-> hand-pobj-> hand-pobj-> in-prep-> in-prep-> get-advcl-> get-advcl-> like-ROOT


hand
hand-pobj-> in-prep-> in-prep-> get-advcl-> get-advcl-> like-ROOT


from
from-prep-> get-advcl-> get-advcl-> like-ROOT


holding
holding-pcomp-> from-prep-> from-prep-> get-advcl-> get-advcl-> like-ROOT


the
the-det-> mouse-dobj-> mouse-dobj-> holding-pcomp-> holding-pcomp-> from-prep-> from-prep-> get-advcl-> get-advcl-> like-ROOT


mouse
mouse-dobj-> holding-pcomp-> holding-pcomp-> from-prep-> from-prep-> get-advcl-> get-advcl-> like-ROOT


,
,-punct-> like-ROOT


I
I-nsubj-> like-ROOT


would
would-aux-> like-ROOT


like



to
to-aux-> have-xcomp-> have-xcomp-> 

In [17]:
h1='They gave the money to whoever presented the winning ticket.'
doc = nlp(h1)

for token in doc:
    print(token)
    print('-> '.join(['{}-{}'.format(dependent_token, dependent_token.dep_) for dependent_token in headList_to_root(token)]))
    print('\n')
doc = nlp(t1)

They
They-nsubj-> gave-ROOT


gave



the
the-det-> money-dobj-> money-dobj-> gave-ROOT


money
money-dobj-> gave-ROOT


to
to-prep-> gave-ROOT


whoever
whoever-nsubj-> presented-pcomp-> presented-pcomp-> to-prep-> to-prep-> gave-ROOT


presented
presented-pcomp-> to-prep-> to-prep-> gave-ROOT


the
the-det-> ticket-dobj-> ticket-dobj-> presented-pcomp-> presented-pcomp-> to-prep-> to-prep-> gave-ROOT


winning
winning-amod-> ticket-dobj-> ticket-dobj-> presented-pcomp-> presented-pcomp-> to-prep-> to-prep-> gave-ROOT


ticket
ticket-dobj-> presented-pcomp-> presented-pcomp-> to-prep-> to-prep-> gave-ROOT


.
.-punct-> gave-ROOT




In [18]:
h1='He was a good boy who helps mother.'
doc = nlp(h1)

for token in doc:
    print(token)
    print('-> '.join(['{}-{}'.format(dependent_token, dependent_token.dep_) for dependent_token in headList_to_root(token)]))
    print('\n')
doc = nlp(t1)

He
He-nsubj-> was-ROOT


was



a
a-det-> boy-attr-> boy-attr-> was-ROOT


good
good-amod-> boy-attr-> boy-attr-> was-ROOT


boy
boy-attr-> was-ROOT


who
who-nsubj-> helps-relcl-> helps-relcl-> boy-attr-> boy-attr-> was-ROOT


helps
helps-relcl-> boy-attr-> boy-attr-> was-ROOT


mother
mother-dobj-> helps-relcl-> helps-relcl-> boy-attr-> boy-attr-> was-ROOT


.
.-punct-> was-ROOT




In [19]:
h1='He has a bag in order to swim.'
doc = nlp(h1)

for token in doc:
    print(token)
    print('-> '.join(['{}-{}'.format(dependent_token, dependent_token.dep_) for dependent_token in headList_to_root(token)]))
    print('\n')
doc = nlp(t1)

He
He-nsubj-> has-ROOT


has



a
a-det-> bag-dobj-> bag-dobj-> has-ROOT


bag
bag-dobj-> has-ROOT


in
in-prep-> has-ROOT


order
order-pobj-> in-prep-> in-prep-> has-ROOT


to
to-aux-> swim-acl-> swim-acl-> order-pobj-> order-pobj-> in-prep-> in-prep-> has-ROOT


swim
swim-acl-> order-pobj-> order-pobj-> in-prep-> in-prep-> has-ROOT


.
.-punct-> has-ROOT




In [20]:
h1='He has a bag which is heavy.'
doc = nlp(h1)

for token in doc:
    print(token)
    print('-> '.join(['{}-{}'.format(dependent_token, dependent_token.dep_) for dependent_token in headList_to_root(token)]))
    print('\n')
doc = nlp(t1)

He
He-nsubj-> has-ROOT


has



a
a-det-> bag-dobj-> bag-dobj-> has-ROOT


bag
bag-dobj-> has-ROOT


which
which-nsubj-> is-relcl-> is-relcl-> bag-dobj-> bag-dobj-> has-ROOT


is
is-relcl-> bag-dobj-> bag-dobj-> has-ROOT


heavy
heavy-acomp-> is-relcl-> is-relcl-> bag-dobj-> bag-dobj-> has-ROOT


.
.-punct-> has-ROOT




In [21]:
h1='He says that you like to swim.'
doc = nlp(h1)

for token in doc:
    print(token)
    print('-> '.join(['{}-{}'.format(dependent_token, dependent_token.dep_) for dependent_token in headList_to_root(token)]))
    print('\n')
doc = nlp(t1)

He
He-nsubj-> says-ROOT


says



that
that-mark-> like-ccomp-> like-ccomp-> says-ROOT


you
you-nsubj-> like-ccomp-> like-ccomp-> says-ROOT


like
like-ccomp-> says-ROOT


to
to-aux-> swim-xcomp-> swim-xcomp-> like-ccomp-> like-ccomp-> says-ROOT


swim
swim-xcomp-> like-ccomp-> like-ccomp-> says-ROOT


.
.-punct-> says-ROOT




In [ ]:
# # test = 'The report of needed classes shall include (but not be limited to) classes to be offered, number of sections needed, number of labs needed, and room types needed'
# # spacy_sent = nlp(test)
# for j, token in enumerate(doc):
#     headlist_to_root = [(dependent_token.dep_) for dependent_token in headList_to_root(token)]
#     headpath_to_root = remove_duplicate(headlist_to_root)  
#     print(high_level_implicit_grouping(headpath_to_root))